# 第3章 设计函数和接口
- 函数
- 多重分派
- 参数化方法
- 接口

## 3.2 设计函数
Julia的工作方式更像是过程/函数式编程语言，而不是面向对象的编程语言。
### 3.2.1 游戏代码

In [1]:
mutable struct Position
    x::Int
    y::Int
end

struct Size
    width::Int
    height::Int
end

struct Widget
    name::String
    position::Position
    size::Size
end

### 3.2.3 定义函数
实际上可以用两种不同语法来定义一个函数：
1. 简单的单行代码，包含函数签名和主体
2. 使用带有签名的function关键字，然后是代码块和end关键字。

In [38]:
# single-line functions
move_up!(widget, v)  = widget.position.y -= v
move_down!(widget, v) = widget.position.y += v
# long version
function move_left!(widget, v)
    widget.position.x -= v
end
function move_right!(widget, v)
    widget.position.x += v
end


move_right! (generic function with 1 method)

#### **函数编写注意事项**
1. **下划线**用于函数名中分隔单词，Julia官方手册约定单词连在一起，不需要分隔，除非太混乱难以阅读，作者认为下划线能增强可读性。
2. **感叹号**用来定义函数能够改变传递到自身的参数。
3. **鸭子类型？**神特么鸭子类型。 在上述函数中，虽然未限制类型，但是希望使用时widget参数具有Widget类型，而v具有Int类型。3.2.3节会讨论。

### 3.2.3 注释函数参数
#### 无类型参数
无类型并不能真正与每种可能的数据类型一起使用，并且，通常可能会导致模糊的异常消息。比如如果将Int值作为Widget参数传递给move_up!函数，那么将报错：Int64没有position字段。这种模糊报错非常不利于调试。

In [4]:
move_up!(1, 2)

ErrorException: type Int64 has no field position

#### 类型化参数
使用一些类型信息来定义函数，通常更安全。比如：

In [39]:
move_up!(widget::Widget, v::Int) = widget.position.y -= v
move_up!(1, 2)

ErrorException: type Int64 has no field position

可以看到，这时再用错误方式调用函数，则会受到一条更加明确的错误信息。
定义以下show函数：

In [40]:
# Define pretty print functions
Base.show(io::IO, p::Position) = print(io, "(", p.x, ",", p.y, ")")
Base.show(io::IO, s::Size) = print(io, s.width, " x ", s.height)
Base.show(io::IO, w::Widget) = print(io, w.name, " at ", w.position, "size", w.size)

测试如下：

In [41]:
# let's test these functions
w = Widget("asteroid", Position(0, 0), Size(10,20))
move_up!(w, 10)
println(w)
move_down!(w, 10)
println(w)
move_left!(w, 10)
println(w)
move_right!(w, 10)
println(w)

asteroid at (0,-10)size10 x 20


asteroid at (0,0)size10 x 20
asteroid at (-10,0)size10 x 20
asteroid at (0,0)size10 x 20


### 3.2.4 使用可选参数
Julia中，可以为参数提供默认值，当具有默认值时，参数将变为可选的。比如，以下是一个随机生成一堆小行星的函数：

In [42]:
# Make a bunch of asteroids
function make_asteroids(N::Int, pos_range = 0:200, size_range = 10:30)
    pos_rand() = rand(pos_range) # 这里为什么要加()，直接定一个pos_rand不可以么？
    sz_rand() = rand(size_range)
    return[Widget("Asteroid #$i", Position(pos_rand(), pos_rand()), Size(sz_rand(), sz_rand()))
    for i in 1:N]
end

make_asteroids (generic function with 3 methods)

可以看到，自动生成三个不同的函数。

In [9]:
make_asteroids(10)

10-element Vector{Widget}:
 Asteroid #1 at (143,3)size21 x 25
 Asteroid #2 at (154,165)size17 x 17
 Asteroid #3 at (47,38)size30 x 16
 Asteroid #4 at (191,54)size30 x 16
 Asteroid #5 at (60,7)size24 x 18
 Asteroid #6 at (179,171)size28 x 24
 Asteroid #7 at (79,40)size11 x 18
 Asteroid #8 at (92,95)size24 x 20
 Asteroid #9 at (1,59)size18 x 26
 Asteroid #10 at (161,72)size21 x 25

In [10]:
make_asteroids(5, 1:10)

5-element Vector{Widget}:
 Asteroid #1 at (10,8)size10 x 12
 Asteroid #2 at (8,3)size16 x 29
 Asteroid #3 at (8,3)size17 x 26
 Asteroid #4 at (5,1)size23 x 27
 Asteroid #5 at (6,2)size19 x 14

In [11]:
make_asteroids(6, 1:10, 1:5)

6-element Vector{Widget}:
 Asteroid #1 at (3,8)size2 x 3
 Asteroid #2 at (6,8)size4 x 4
 Asteroid #3 at (9,1)size5 x 4
 Asteroid #4 at (6,9)size2 x 3
 Asteroid #5 at (9,9)size3 x 1
 Asteroid #6 at (3,7)size2 x 5

上述代码后面两个参数很容易混掉，调用时必须要注意定义顺序，参数更多时，可读性会变差。因此，可以使用关键字参数。

### 3.2.5 使用关键字参数
与之前函数的唯一区别是一个字符，位置参数和关键字参数用;分号分隔，而不再是逗号。

In [12]:
function make_asteroids2(N::Int; pos_range = 0:200, size_range = 10:30)
    pos_rand() = rand(pos_range) # 这里为什么要加()，直接定一个pos_rand不可以么？
    sz_rand() = rand(size_range)
    return[Widget("Asteroid #$i", Position(pos_rand(), pos_rand()), Size(sz_rand(), sz_rand()))
    for i in 1:N]
end

make_asteroids2 (generic function with 1 method)

此时，再调时关键字参数则必须与参数名称一起传递，并且，不再需要考虑顺序。

In [13]:
make_asteroids2(5)

5-element Vector{Widget}:
 Asteroid #1 at (65,29)size28 x 13
 Asteroid #2 at (48,8)size15 x 20
 Asteroid #3 at (99,86)size21 x 29
 Asteroid #4 at (92,73)size28 x 15
 Asteroid #5 at (137,22)size19 x 30

In [16]:
make_asteroids2(5, size_range = 1:5)

5-element Vector{Widget}:
 Asteroid #1 at (197,199)size5 x 2
 Asteroid #2 at (114,155)size3 x 4
 Asteroid #3 at (135,174)size2 x 3
 Asteroid #4 at (114,38)size3 x 2
 Asteroid #5 at (111,103)size2 x 5

另一个特性是关键字参数不必带有任何默认值，比如：

In [17]:
function make_asteroids3(; N::Int, pos_range = 0:200, size_range = 10:30)
    pos_rand() = rand(pos_range) # 这里为什么要加()，直接定一个pos_rand不可以么？
    sz_rand() = rand(size_range)
    return[Widget("Asteroid #$i", Position(pos_rand(), pos_rand()), Size(sz_rand(), sz_rand()))
    for i in 1:N]
end

make_asteroids3 (generic function with 1 method)

因为第一个参数N未设置默认值，因此N成为强制性关键字参数，此时可以调用指定N的函数。

In [21]:
asteroids = make_asteroids3(N = 5)

5-element Vector{Widget}:
 Asteroid #1 at (59,133)size10 x 22
 Asteroid #2 at (102,174)size17 x 21
 Asteroid #3 at (120,70)size18 x 29
 Asteroid #4 at (169,22)size18 x 26
 Asteroid #5 at (105,32)size30 x 23

当一个函数需要许多参数时，它也可以很好地工作。

### 3.2.6 接受可变数量的参数
在函数参数中添加三个点，则会自动将所有传递的参数汇总到一个变量中。此功能称为slurping。

In [22]:
# Shoot any number of targets
function shoot(from::Widget, targets::Widget...)
    println("Type of targets: ", typeof(targets))
    for target in targets
        println(from.name, "--> ", target.name)
    end
end

shoot (generic function with 1 method)

In [23]:
spaceship = Widget("Spaceship", Position(0, 0), Size(30, 30))
target1 = asteroids[1]
target2 = asteroids[2]
target3 = asteroids[3]

Asteroid #3 at (120,70)size18 x 29

In [24]:
shoot(spaceship, target1)

Type of targets: Tuple{

Widget}
Spaceship--> Asteroid #1


In [26]:
shoot(spaceship, target1, target3, target2)

Type of targets: Tuple{Widget, Widget, Widget}
Spaceship--> Asteroid #1
Spaceship--> Asteroid #3
Spaceship--> Asteroid #2


### 3.2.7 splatting参数
调用函数时，当变量后面跟三个点时，则该变量自动分配为多个函数参数，此功能成为splatting。比如，下面一个例子，以特定形式排列几个飞船：

In [27]:
# Special arrangement before attacks
function triangular_formation!(s1::Widget, s2::Widget, s3::Widget)
    x_offset = 30
    y_offset = 50
    s2.position.x = s1.position.x - x_offset
    s3.position.x = s1.position.x + x_offset
    s2.position.y = s3.position.y = s1.position.y - y_offset
    (s1, s2, s3)
end

triangular_formation! (generic function with 1 method)

先建造几个飞船

In [76]:
spaceships = [Widget("Spaceship $i", Position(0, 0), Size(20, 50))
for i in 1:3
    ]

3-element Vector{Widget}:
 Spaceship 1 at (0,0)size20 x 50
 Spaceship 2 at (0,0)size20 x 50
 Spaceship 3 at (0,0)size20 x 50

In [68]:
triangular_formation!(spaceships...)

(Spaceship 1 at (0,0)size20 x 50, Spaceship 2 at (-30,-50)size20 x 50, Spaceship 3 at (30,-50)size20 x 50)

可以看到，spaceship数组里面的三个元素被分配到三个参数以满足triangular_formation!函数的期望。

### 3.2.8 第一类实体函数
当函数可以分配给变量或结构字段、传递给函数、从函数返回时，它们被认为是第一类实体。
设计一个新函数，可以让飞船沿随机方向飞行一段随机距离。

In [36]:
function random_move()
    return rand([move_up!, move_down!, move_left!, move_right!])
end

function random_leap!(w::Widget, move_func::Function, distance::Int)
    move_func(w, distance)
    return w
end

random_leap! (generic function with 1 method)

In [74]:
spaceship = Widget("Spaceship", Position(0, 0), Size(20, 50))

Spaceship at (0,0)size20 x 50

In [75]:
random_leap!(spaceship, random_move(), rand(50:100))

Spaceship at (-92,0)size20 x 50

### 3.2.9 开发匿名函数
有时，只想创建一个简单的函数并在不分配名称的情况下传递它。这种编程风格实际上在函数编程语言中相当普遍。比如，我们要炸毁所有小行星，一种方法是定义一个explode函数，并传递给foreach函数，例如：

In [60]:
function explode(x)
    println(x, " exploded!")
end

function clean_up_galaxy(asteroids)
    foreach(explode, asteroids)
end

clean_up_galaxy (generic function with 1 method)

In [61]:
clean_up_galaxy(asteroids)

Asteroid #1 at (59,133)size10 x 22 exploded!
Asteroid #2 at (102,174)size17 x 21 exploded!
Asteroid #3 at (120,70)size18 x 29 exploded!
Asteroid #4 at (169,22)size18 x 26 exploded!
Asteroid #5 at (105,32)size30 x 23 exploded!


实现了想要的功能。

如果仅将及iiniminghanshu传递给forech，我们可以达到相同的效果。

In [62]:
function clean_up_galaxy1(asteroids)
    foreach(x -> println(x, "exploded"), asteroids)
end

clean_up_galaxy1 (generic function with 1 method)

In [63]:
clean_up_galaxy1(asteroids)

Asteroid #1 at (59,133)size10 x 22exploded
Asteroid #2 at (102,174)size17 x 21exploded
Asteroid #3 at (120,70)size18 x 29exploded
Asteroid #4 at (169,22)size18 x 26exploded
Asteroid #5 at (105,32)size30 x 23exploded


匿名函数的语法包含参数变量，后跟箭头和函数主体。这种情况下，我们只有一个参数。如果有更多的参数，可以将它们编写为包含在括号中的元组。匿名函数也可以分配给变量并传递。

比如也要炸飞船，则

In [71]:
function clean_up_galaxy(asteroids, spaceships)
    ep = x -> println(x, " exploded!")
    foreach(ep, asteroids)
    foreach(ep, spaceships)
end

clean_up_galaxy (generic function with 2 methods)

In [77]:
clean_up_galaxy(asteroids, spaceships)

Asteroid #1 at (59,133)size10 x 22 exploded!
Asteroid #2 at (102,174)size17 x 21 exploded!
Asteroid #3 at (120,70)size18 x 29 exploded!
Asteroid #4 at (169,22)size18 x 26 exploded!
Asteroid #5 at (105,32)size30 x 23 exploded!
Spaceship 1 at (0,0)size20 x 50 exploded!
Spaceship 2 at (0,0)size20 x 50 exploded!
Spaceship 3 at (0,0)size20 x 50 exploded!


#### 匿名函数的优点
- 无需使用函数名称并污染模块的命名空间。
- 在调用时提供匿名函数逻辑使代码更易于月的。
- 代码更加紧凑。

### 3.2.10 使用do语法